In [1]:
# Install python package for cassandra:
!pip install cassandra-driver

DEPRECATION: Loading egg at /opt/bitnami/python/lib/python3.11/site-packages/pip-23.3.2-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 44.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 14.1 MB/s eta 0:00:00


In [43]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

auth_provider = PlainTextAuthProvider('cassandra', 'cassandra')
cluster = Cluster(['cassandra', 'localhost'], auth_provider=auth_provider)
session = cluster.connect()
session.set_keyspace("opensea")
rows = session.execute("SELECT * FROM opensea.global_metrics")
rows

NoHostAvailable: ('Unable to connect to any servers', {'::1:9042': ConnectionRefusedError(111, "Tried connecting to [('::1', 9042, 0, 0)]. Last error: Connection refused"), '127.0.0.1:9042': ConnectionRefusedError(111, "Tried connecting to [('127.0.0.1', 9042)]. Last error: Connection refused")})

Exception in thread Task Scheduler:
Traceback (most recent call last):
  File "/opt/bitnami/python/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
Exception in thread Task Scheduler:
Traceback (most recent call last):
  File "/opt/bitnami/python/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
Exception in thread Task Scheduler:
Traceback (most recent call last):
  File "/opt/bitnami/python/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "cassandra/cluster.py", line 4239, in cassandra.cluster._Scheduler.run
    self.run()
  File "cassandra/cluster.py", line 4239, in cassandra.cluster._Scheduler.run
Exception in thread Task Scheduler:
Traceback (most recent call last):
  File "/opt/bitnami/python/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "cassandra/cluster.py", line 4239, in cassandra.cluster._Scheduler.run
    self.run()
  File "cassandra/cluster.py", line 4239, in cassandra.cluster._Sc

In [ ]:
result = [row for row in rows]
print(len(result))
result

580


[Row(metric='transfers_count__1_minute', timestamp=datetime.datetime(2024, 7, 20, 19, 5, 30), collection=None, value=152.0),
 Row(metric='transfers_count__1_minute', timestamp=datetime.datetime(2024, 7, 20, 19, 5), collection=None, value=84.0),
 Row(metric='transfers_count__1_minute', timestamp=datetime.datetime(2024, 7, 20, 19, 4, 30), collection=None, value=126.0),
 Row(metric='transfers_count__1_minute', timestamp=datetime.datetime(2024, 7, 20, 19, 4), collection=None, value=138.0),
 Row(metric='transfers_count__1_minute', timestamp=datetime.datetime(2024, 7, 20, 19, 3, 30), collection=None, value=176.0),
 Row(metric='transfers_count__1_minute', timestamp=datetime.datetime(2024, 7, 20, 19, 3), collection=None, value=211.0),
 Row(metric='transfers_count__1_minute', timestamp=datetime.datetime(2024, 7, 20, 19, 2, 30), collection=None, value=187.0),
 Row(metric='transfers_count__1_minute', timestamp=datetime.datetime(2024, 7, 20, 19, 1, 30), collection=None, value=1450.0),
 Row(metric=